In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

# Downloading the python small dataset
dataset = load_dataset("Fsoft-AIC/the-vault-function", split_set=["train/small"], languages=['python'],trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

the-vault-function.py:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

python-00000-of-00002.parquet:   0%|          | 0.00/278M [00:00<?, ?B/s]

python-00001-of-00002.parquet:   0%|          | 0.00/276M [00:00<?, ?B/s]

Generating train_small split: 0 examples [00:00, ? examples/s]

In [3]:
dataset['train_small'][0].keys()

dict_keys(['hexsha', 'repo', 'path', 'license', 'language', 'identifier', 'return_type', 'original_string', 'original_docstring', 'docstring', 'docstring_tokens', 'code', 'code_tokens', 'short_docstring', 'short_docstring_tokens', 'comment', 'parameters', 'docstring_params'])

In [4]:
print(dataset['train_small'][7]["original_string"])

async def handle(self, context: RequestContext, responder: BaseResponder):
        """
        Message handler logic for credential requests.

        Args:
            context: request context
            responder: responder callback

        """
        r_time = get_timer()

        self._logger.debug("V20CredRequestHandler called with context %s", context)
        assert isinstance(context.message, V20CredRequest)
        self._logger.info(
            "Received v2.0 credential request message: %s",
            context.message.serialize(as_string=True),
        )

        if not context.connection_ready:
            raise HandlerException("No connection established for credential request")

        cred_manager = V20CredManager(context.profile)
        cred_ex_record = await cred_manager.receive_request(
            context.message, context.connection_record.connection_id
        )  # mgr only finds, saves record: on exception, saving state null is hopeless

        r_time = trace_

In [5]:
from datasets import Dataset

# Extract the conversations
def create_conversation_only(example):
    comment = example.get('docstring', '') or ''  # Handle missing or None values
    code = example.get('original_string', '') or ''
    return [
        {"role": "user", "content": comment},
        {"role": "assistant", "content": code}
    ]

# Create a list of conversations
conversations = [
    {"conversation": create_conversation_only(example)}
    for example in dataset['train_small']
]

In [6]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [7]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.2: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.12.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [9]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

def formatt(conversations):
    text = []
    for convos in conversations:
        text.append([tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos['conversation']])
    return conversations

In [11]:
import pandas as pd

# Ici j'ai bidouller pour avoir la bonne entrée en dataset
df = pd.DataFrame(conversations)
dataset_list = df['conversation'].to_list()

def transform_to_chat_format(conversation):
    """
    Transforms a conversation from a list of dictionaries with 'content' and 'role'
    into the specified string format.
    """
    output = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
    output += "Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|>"

    for message in conversation:
        role = message['role']
        content = message['content']
        output += f"<|start_header_id|>{role}<|end_header_id|>\n\n{content}\n<|eot_id|>"

    return output

text_list = []
for elem in dataset_list:
    text_list.append(transform_to_chat_format(elem))
dataset = Dataset.from_dict({'text': text_list})

In [12]:
print(dataset[9]['text'])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Message handler logic for credential offers.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

async def handle(self, context: RequestContext, responder: BaseResponder):
        """
        Message handler logic for credential offers.

        Args:
            context: request context
            responder: responder callback

        """
        r_time = get_timer()

        self._logger.debug("V20CredIssueHandler called with context %s", context)
        assert isinstance(context.message, V20CredIssue)
        self._logger.info(
            "Received v2.0 credential issue message: %s",
            context.message.serialize(as_string=True),
        )

        if not context.connection_ready:
            raise HandlerException("No connection established for credential issue")

        cred_manager = V20C

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/370657 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [14]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/370657 [00:00<?, ? examples/s]

In [15]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nMessage handler logic for v2.0 presentation requests.\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nasync def handle(self, context: RequestContext, responder: BaseResponder):\n        """\n        Message handler logic for v2.0 presentation requests.\n\n        Args:\n            context: request context\n            responder: responder callback\n\n        """\n        r_time = get_timer()\n\n        self._logger.debug("V20PresRequestHandler called with context %s", context)\n        assert isinstance(context.message, V20PresRequest)\n        self._logger.info(\n            "Received v2.0 presentation request message: %s",\n            context.message.serialize(as_string=True),\n        )\n\n        if not context.connection_ready:\n            raise HandlerException("No connection establi

In [16]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                             \n\nasync def handle(self, context: RequestContext, responder: BaseResponder):\n        """\n        Message handler logic for v2.0 presentation requests.\n\n        Args:\n            context: request context\n            responder: responder callback\n\n        """\n        r_time = get_timer()\n\n        self._logger.debug("V20PresRequestHandler called with context %s", context)\n        assert isinstance(context.message, V20PresRequest)\n        self._logger.info(\n            "Received v2.0 presentation request message: %s",\n            context.message.serialize(as_string=True),\n        )\n\n        if not context.connection_ready:\n            raise HandlerException("No connection established for presentation request")\n\n        pres_manager = V20PresManager(context.profile)\n\n        # Get pres ex record (holder initiated via proposal)\n        # or create it (verifier sent request first)\n        try:\n            async with con

In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 370,657 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,1.638000
2,1.619400
3,1.699800
4,1.268400
5,1.256400
6,1.525800
7,1.493800
8,1.668500
9,1.529700
10,1.565400


In [18]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "write the code for a full transformer model in pytorch"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
text = tokenizer.batch_decode(outputs)
text

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nwrite the code for a full transformer model in pytorch<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\ndef create_full_transformer(model_name: str) -> nn.Module:\n        """\n        Create a full Transformer model, which is equivalent to Transformer-XL.\n\n        Args:\n            model_name (str): Name of the model.\n        """\n\n        def embedding(input_len, hidden_size, seq_len):\n            """Embedding layer."""\n            embedding_layer = nn.Embedding(input_len, hidden_size)\n            # (seq_len + padding + dim) -> (seq_len, dim)\n            embedding_layer.weight.requires_grad_(True)\n            embedding = nn.utils.weight_divide(embedding_layer.weight, input_len)\n\n            return nn.utils.rnn.pack_sequence(\n                embedding,\n                enforce_sor

In [19]:
print(text[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

write the code for a full transformer model in pytorch<|eot_id|><|start_header_id|>assistant<|end_header_id|>

def create_full_transformer(model_name: str) -> nn.Module:
        """
        Create a full Transformer model, which is equivalent to Transformer-XL.

        Args:
            model_name (str): Name of the model.
        """

        def embedding(input_len, hidden_size, seq_len):
            """Embedding layer."""
            embedding_layer = nn.Embedding(input_len, hidden_size)
            # (seq_len + padding + dim) -> (seq_len, dim)
            embedding_layer.weight.requires_grad_(True)
            embedding = nn.utils.weight_divide(embedding_layer.weight, input_len)

            return nn.utils.rnn.pack_sequence(
                embedding,
                enforce_sorted=False,
                e

In [21]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [22]:
!zip -r lora_model.zip lora_model

  adding: lora_model/ (stored 0%)
  adding: lora_model/tokenizer.json (deflated 85%)
  adding: lora_model/adapter_config.json (deflated 54%)
  adding: lora_model/README.md (deflated 66%)
  adding: lora_model/tokenizer_config.json (deflated 94%)
  adding: lora_model/adapter_model.safetensors (deflated 8%)
  adding: lora_model/special_tokens_map.json (deflated 71%)
